In [2]:
#import pandas and numpy libraries
import pandas as pd
import numpy as np
import sys #sys needed only for python version
#import gaussian naive bayes from scikit-learn
import sklearn as sk
#seaborn for pretty plots
import seaborn as sns

#display versions of python and packages
print('\npython version ' + sys.version)
print('pandas version ' + pd.__version__)
print('numpy version ' + np.__version__)
print('sk-learn version ' + sk.__version__)
print('seaborn version ' + sns.__version__)


python version 3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
pandas version 0.20.3
numpy version 1.12.1
sk-learn version 0.19.0
seaborn version 0.8.0


In [6]:
#read in data. it's comma-separated with no column names.
df = pd.read_csv('CustomerInnovationSurveyDS-Data.csv' )
# set pandas to output all of the columns in output
pd.options.display.max_columns = 26
#show the first 5 rows
print(df.sample(n=5))

     Accounts Manager Identification Number  Customer Identification Number  \
57                                        3                              57   
12                                        0                              12   
42                                        3                              42   
31                                        3                              31   
129                                      10                             129   

     Innovative  Relationship Rating  Visited Tech Center (0=NO, 1=YES)  \
57            1                    1                                  0   
12            1                    2                                  0   
42            1                    2                                  0   
31            1                    2                                  0   
129           5                    5                                  1   

     ALPHA (0=not interested, 1=yes interested, 2=Maybe)  \
57            

In [4]:
#add column names from documentation (1st col is class: e=edible,p=poisonous; rest are attributes)
df.columns = ['Account Manager Identification Number','Customer Identification Number','Innovative','Relationship Rating',
             'Visited Tech Center','ALPHA','BETA','PILOT','have they done pilot with us',
             'Imporve occupancy via differentiation/positioning','Increase rate/bed through add-on services','Increase 5-star rating','Staff efficiency / lower cost of delivering care',
             'Insurance costs','Utilities costs','Health & Wellness Clinicial Rehab','Resident monitoring and saftey','Rehospitalization/ACo partnerships','Care coordination across settings','Food & Nutirition', 'Telehealth and remote care delivery', 'Rural health and wellness','Number of pilots in progress']

print("Example values:\n")
print(df.iloc[20])

Example values:

Account Manager Identification Number                  1
Customer Identification Number                        20
Innovative                                             2
Relationship Rating                                    3
Visited Tech Center                                    0
ALPHA                                                  0
BETA                                                   0
PILOT                                                  0
have they done pilot with us                           0
Imporve occupancy via differentiation/positioning      3
Increase rate/bed through add-on services            NaN
Increase 5-star rating                                 2
Staff efficiency / lower cost of delivering care       1
Insurance costs                                      NaN
Utilities costs                                      NaN
Health & Wellness Clinicial Rehab                      4
Resident monitoring and saftey                       NaN
Rehospitalizat

In [5]:
#show plots in notebook
%matplotlib inline

#bar chart of classes using pandas plotting
print(df['class'].value_counts())
df['class'].value_counts().plot(kind='scatter')

KeyError: 'class'

In [ ]:
#seaborn factorplot to show edible/poisonous breakdown by different factors
df_forplot = df.loc[:,('class','cap-shape','gill-color')]
g = sns.factorplot("class", col="cap-shape", data=df_forplot,
                    kind="count", size=2.5, aspect=.8, col_wrap=6)

In [ ]:
g = sns.factorplot("class", col="gill-color", data=df_forplot,
                    kind="count", size=2.5, aspect=.8, col_wrap=6)

In [9]:
#put the features into X (everything except the 0th column)
X = pd.DataFrame(df, columns=df.columns[1:len(df.columns)], index=df.index)
#put the class values (0th column) into Y 
Y = df['class']

#encode the text category labels as numeric
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
#print(le.classes_)
#print(np.array(Y))
#Y values now boolean values; poison = 1
y = le.transform(Y)
#print(y_train)

#have to initialize or get error below
x = pd.DataFrame(X,columns=[X.columns[0]])
#encode each feature column and add it to x_train
for colname in X.columns:
    le.fit(X[colname])
    print(colname, le.classes_)
    x[colname] = le.transform(X[colname])

print('\nExample Feature Values - row 1 in X:')
print(X.iloc[1])
print('\nExample Encoded Feature Values - row 1 in x:')
print(x.iloc[1])
print('\nClass Values (Y):')
print(np.array(Y))
print('\nEncoded Class Values (y):')
print(y)

cap-shape ['b' 'c' 'f' 'k' 's' 'x']
cap-surface ['f' 'g' 's' 'y']
cap-color ['b' 'c' 'e' 'g' 'n' 'p' 'r' 'u' 'w' 'y']
bruises ['f' 't']
odor ['a' 'c' 'f' 'l' 'm' 'n' 'p' 's' 'y']
gill-attachment ['a' 'f']
gill-spacing ['c' 'w']
gill-size ['b' 'n']
gill-color ['b' 'e' 'g' 'h' 'k' 'n' 'o' 'p' 'r' 'u' 'w' 'y']
stalk-shape ['e' 't']
stalk-root ['?' 'b' 'c' 'e' 'r']
stalk-surf-above-ring ['f' 'k' 's' 'y']
stalk-surf-below-ring ['f' 'k' 's' 'y']
stalk-color-above-ring ['b' 'c' 'e' 'g' 'n' 'o' 'p' 'w' 'y']
stalk-color-below-ring ['b' 'c' 'e' 'g' 'n' 'o' 'p' 'w' 'y']
veil-type ['p']
veil-color ['n' 'o' 'w' 'y']
ring-number ['n' 'o' 't']
ring-type ['e' 'f' 'l' 'n' 'p']
spore-color ['b' 'h' 'k' 'n' 'o' 'r' 'u' 'w' 'y']
population ['a' 'c' 'n' 's' 'v' 'y']
habitat ['d' 'g' 'l' 'm' 'p' 'u' 'w']

Example Feature Values - row 1 in X:
cap-shape                 x
cap-surface               s
cap-color                 y
bruises                   t
odor                      a
gill-attachment           f


In [10]:
# actually doing the damn machine learning
#split the dataset into training and test sets
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

#initialize and fit the naive bayes classifier
from sklearn.naive_bayes import GaussianNB
skgnb = GaussianNB()
skgnb.fit(x_train,y_train)
train_predict = skgnb.predict(x_train)
#print(train_predict)

#see how accurate the training data was fit
from sklearn import metrics
print("Training accuracy:",metrics.accuracy_score(y_train, train_predict))

#use the trained model to predict the test values
test_predict = skgnb.predict(x_test)
print("Testing accuracy:",metrics.accuracy_score(y_test, test_predict))

Training accuracy: 0.925592504134
Testing accuracy: 0.916076091011


In [12]:
print("\nClassification Report:")
print(metrics.classification_report(y_test, test_predict, target_names=['edible','poisonous']))
print("\nConfusion Matrix:")
skcm = metrics.confusion_matrix(y_test,test_predict)
#putting it into a dataframe so it prints the labels
skcm = pd.DataFrame(skcm, columns=['predicted-edible','predicted-poisonous'])
skcm['actual'] = ['edible','poisonous']
skcm = skcm.set_index('actual')


print(skcm)

print("\nScore (same thing as test accuracy?): ", skgnb.score(x_test,y_test))


Classification Report:
             precision    recall  f1-score   support

     edible       0.92      0.92      0.92      1382
  poisonous       0.91      0.91      0.91      1299

avg / total       0.92      0.92      0.92      2681


Confusion Matrix:
           predicted-edible  predicted-poisonous
actual                                          
edible                 1269                  113
poisonous               112                 1187

Score (same thing as test accuracy?):  0.916076091011
